In [3]:
# Cài đặt numpy
!pip install numpy

# Cài đặt pandas
!pip install pandas

# Nếu muốn test thử có thể cài thêm matplotlib để visualize (không bắt buộc)
!pip install matplotlib

In [4]:
import numpy as np
import pandas as pd
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)

NumPy version: 2.1.3
Pandas version: 2.2.3


In [15]:
import numpy as np
import pandas as pd
import math
import random

# Cấu hình hiển thị pandas cho dễ nhìn
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Đã import thư viện!")

Đã import thư viện!


In [16]:
#chuẩn hóa (Scaling)
class Normalizer:
    """Chuẩn hóa dữ liệu về khoảng [0, 1] (Min-Max Scaling)"""
    
    def __init__(self):
        self.min_vals = None
        self.max_vals = None
        
    def fit(self, X):
        """Tính min và max của từng cột"""
        if isinstance(X, pd.DataFrame):
            X = X.values
            
        # Lưu min, max của từng cột
        self.min_vals = np.min(X, axis=0)
        self.max_vals = np.max(X, axis=0)
        
        # Tránh chia cho 0 (nếu min = max)
        for i in range(len(self.max_vals)):
            if self.max_vals[i] == self.min_vals[i]:
                self.max_vals[i] = 1
                
        return self
    
    def transform(self, X):
        """Áp dụng công thức: (x - min) / (max - min)"""
        if isinstance(X, pd.DataFrame):
            X_np = X.values.copy()
            is_dataframe = True
        else:
            X_np = X.copy()
            is_dataframe = False
            
        # Kiểm tra đã fit chưa
        if self.min_vals is None or self.max_vals is None:
            raise ValueError("Cần gọi fit() trước khi transform()")
            
        # Áp dụng normalization
        for i in range(X_np.shape[1]):
            denominator = self.max_vals[i] - self.min_vals[i]
            if denominator != 0:
                X_np[:, i] = (X_np[:, i] - self.min_vals[i]) / denominator
            else:
                X_np[:, i] = 0
                
        # Trả về định dạng ban đầu
        if is_dataframe:
            return pd.DataFrame(X_np, columns=X.columns)
        return X_np
    
    def fit_transform(self, X):
        """Gộp fit và transform"""
        self.fit(X)
        return self.transform(X) 

class StandardScaler:
    """Chuẩn hóa dữ liệu về mean=0, std=1 (Z-Score Scaling)"""
    
    def __init__(self):
        self.mean_vals = None
        self.std_vals = None
        
    def fit(self, X):
        """Tính mean và std của từng cột"""
        if isinstance(X, pd.DataFrame):
            X = X.values
            
        self.mean_vals = np.mean(X, axis=0)
        self.std_vals = np.std(X, axis=0)
        
        # Tránh chia cho 0
        for i in range(len(self.std_vals)):
            if self.std_vals[i] == 0:
                self.std_vals[i] = 1
                
        return self
    
    def transform(self, X):
        """Áp dụng công thức: (x - mean) / std"""
        if isinstance(X, pd.DataFrame):
            X_np = X.values.copy()
            is_dataframe = True
        else:
            X_np = X.copy()
            is_dataframe = False
            
        if self.mean_vals is None or self.std_vals is None:
            raise ValueError("Cần gọi fit() trước khi transform()")
            
        # Áp dụng standardization
        for i in range(X_np.shape[1]):
            if self.std_vals[i] != 0:
                X_np[:, i] = (X_np[:, i] - self.mean_vals[i]) / self.std_vals[i]
            else:
                X_np[:, i] = 0
                
        if is_dataframe:
            return pd.DataFrame(X_np, columns=X.columns)
        return X_np
    
    def fit_transform(self, X):
        """Gộp fit và transform"""
        self.fit(X)
        return self.transform(X)

In [7]:
class StandardScaler:
    """Chuẩn hóa dữ liệu về mean=0, std=1"""
    
    def __init__(self):
        self.mean_vals = None
        self.std_vals = None
        
    def fit(self, X):
        """Tính mean và std của từng cột"""
        if isinstance(X, pd.DataFrame):
            X = X.values
            
        self.mean_vals = np.mean(X, axis=0)
        self.std_vals = np.std(X, axis=0)
        
        # Tránh chia cho 0
        for i in range(len(self.std_vals)):
            if self.std_vals[i] == 0:
                self.std_vals[i] = 1
                
        return self
    
    def transform(self, X):
        """Áp dụng công thức: (x - mean) / std"""
        if isinstance(X, pd.DataFrame):
            X_np = X.values.copy()
            is_dataframe = True
        else:
            X_np = X.copy()
            is_dataframe = False
            
        if self.mean_vals is None or self.std_vals is None:
            raise ValueError("Cần gọi fit() trước khi transform()")
            
        # Áp dụng standardization
        for i in range(X_np.shape[1]):
            if self.std_vals[i] != 0:
                X_np[:, i] = (X_np[:, i] - self.mean_vals[i]) / self.std_vals[i]
            else:
                X_np[:, i] = 0
                
        if is_dataframe:
            return pd.DataFrame(X_np, columns=X.columns)
        return X_np
    
    def fit_transform(self, X):
        """Gộp fit và transform"""
        self.fit(X)
        return self.transform(X)

In [17]:
class MissingValueHandler:
    """Xử lý giá trị thiếu (Shipper dọn rác)"""
    
    @staticmethod
    def detect_missing(df):
        return df.isnull()
    
    @staticmethod
    def count_missing(df):
        return df.isnull().sum()
    
    @staticmethod
    def fill_mean(df, columns=None):
        """Điền missing bằng giá trị trung bình"""
        df_copy = df.copy()
        
        if columns is None:
            columns = df_copy.select_dtypes(include=[np.number]).columns
            
        for col in columns:
            if col in df_copy.columns:
                mean_val = df_copy[col].mean()
                # ✅ ĐÃ FIX: Gán trực tiếp, ko dùng inplace=True
                df_copy[col] = df_copy[col].fillna(mean_val)
                
        return df_copy
    
    @staticmethod
    def fill_median(df, columns=None):
        """Điền missing bằng giá trị trung vị"""
        df_copy = df.copy()
        
        if columns is None:
            columns = df_copy.select_dtypes(include=[np.number]).columns
            
        for col in columns:
            if col in df_copy.columns:
                median_val = df_copy[col].median()
                # ✅ ĐÃ FIX
                df_copy[col] = df_copy[col].fillna(median_val)
                
        return df_copy
    
    @staticmethod
    def fill_mode(df, columns=None):
        """Điền missing bằng giá trị xuất hiện nhiều nhất"""
        df_copy = df.copy()
        
        if columns is None:
            columns = df_copy.columns
            
        for col in columns:
            if col in df_copy.columns:
                mode_vals = df_copy[col].mode()
                if not mode_vals.empty:
                    # ✅ ĐÃ FIX
                    df_copy[col] = df_copy[col].fillna(mode_vals[0])
                    
        return df_copy
    
    @staticmethod
    def fill_value(df, value, columns=None):
        """Điền missing bằng giá trị cố định"""
        df_copy = df.copy()
        
        if columns is None:
            columns = df_copy.columns
            
        for col in columns:
            if col in df_copy.columns:
                # ✅ ĐÃ FIX
                df_copy[col] = df_copy[col].fillna(value)
                
        return df_copy
    
    @staticmethod
    def drop_rows(df, thresh=None):
        if thresh is None:
            return df.dropna()
        else:
            return df.dropna(thresh=thresh)
    
    @staticmethod
    def drop_columns(df, threshold=0.5):
        missing_ratio = df.isnull().sum() / len(df)
        cols_to_drop = missing_ratio[missing_ratio > threshold].index
        return df.drop(columns=cols_to_drop)

class CategoricalEncoder:
    """Mã hóa biến phân loại (Phiên dịch viên)"""
    
    @staticmethod
    def one_hot_encode(df, columns=None, drop_first=False):
        df_copy = df.copy()
        
        if columns is None:
            cat_cols = df_copy.select_dtypes(include=['object', 'category']).columns
        else:
            cat_cols = columns
            
        num_cols = [col for col in df_copy.columns if col not in cat_cols]
        result_dfs = []
        
        if num_cols:
            result_dfs.append(df_copy[num_cols])
            
        for col in cat_cols:
            if col in df_copy.columns:
                unique_vals = df_copy[col].dropna().unique()
                for val in unique_vals:
                    new_col_name = f"{col}_{val}"
                    new_col_data = (df_copy[col] == val).astype(int)
                    result_dfs.append(pd.DataFrame({new_col_name: new_col_data}))
        
        result = pd.concat(result_dfs, axis=1)
        
        if drop_first and len(cat_cols) > 0:
             # Logic drop first đơn giản hóa
             pass
                    
        return result
    
    @staticmethod
    def label_encode(df, columns=None):
        df_copy = df.copy()
        
        if columns is None:
            cat_cols = df_copy.select_dtypes(include=['object', 'category']).columns
        else:
            cat_cols = columns
            
        encoding_maps = {}
        
        for col in cat_cols:
            if col in df_copy.columns:
                unique_vals = df_copy[col].dropna().unique()
                encoding_map = {val: i for i, val in enumerate(unique_vals)}
                df_copy[col] = df_copy[col].map(encoding_map)
                encoding_maps[col] = encoding_map
                
        return df_copy, encoding_maps
    
    @staticmethod
    def frequency_encode(df, columns=None):
        df_copy = df.copy()
        if columns is None:
            cat_cols = df_copy.select_dtypes(include=['object', 'category']).columns
        else:
            cat_cols = columns
            
        for col in cat_cols:
            if col in df_copy.columns:
                freq = df_copy[col].value_counts(normalize=True)
                df_copy[col] = df_copy[col].map(freq)
                
        return df_copy

In [9]:
class CategoricalEncoder:
    """Mã hóa biến phân loại"""
    
    @staticmethod
    def one_hot_encode(df, columns=None, drop_first=False):
        """
        One-hot encoding
        drop_first: có bỏ cột đầu tiên không (tránh multicollinearity)
        """
        df_copy = df.copy()
        
        if columns is None:
            # Tự động tìm cột categorical
            cat_cols = df_copy.select_dtypes(include=['object', 'category']).columns
        else:
            cat_cols = columns
            
        # Lưu các cột số để giữ lại
        num_cols = [col for col in df_copy.columns if col not in cat_cols]
        result_dfs = []
        
        # Thêm cột số trước
        if num_cols:
            result_dfs.append(df_copy[num_cols])
            
        # Xử lý từng cột categorical
        for col in cat_cols:
            if col in df_copy.columns:
                unique_vals = df_copy[col].dropna().unique()
                
                # Tạo cột mới cho mỗi giá trị unique
                for val in unique_vals:
                    new_col_name = f"{col}_{val}"
                    new_col_data = (df_copy[col] == val).astype(int)
                    result_dfs.append(pd.DataFrame({new_col_name: new_col_data}))
        
        # Ghép tất cả lại
        result = pd.concat(result_dfs, axis=1)
        
        # Bỏ cột đầu nếu cần
        if drop_first and cat_cols.any():
            for col in cat_cols:
                if f"{col}_{df_copy[col].iloc[0]}" in result.columns:
                    result = result.drop(f"{col}_{df_copy[col].iloc[0]}", axis=1)
                    
        return result
    
    @staticmethod
    def label_encode(df, columns=None):
        """
        Label encoding - gán số cho mỗi giá trị unique
        Trả về DataFrame đã encode và mapping dictionary
        """
        df_copy = df.copy()
        
        if columns is None:
            cat_cols = df_copy.select_dtypes(include=['object', 'category']).columns
        else:
            cat_cols = columns
            
        encoding_maps = {}
        
        for col in cat_cols:
            if col in df_copy.columns:
                # Tạo mapping từ giá trị sang số
                unique_vals = df_copy[col].dropna().unique()
                encoding_map = {val: i for i, val in enumerate(unique_vals)}
                
                # Áp dụng mapping
                df_copy[col] = df_copy[col].map(encoding_map)
                encoding_maps[col] = encoding_map
                
        return df_copy, encoding_maps
    
    @staticmethod
    def frequency_encode(df, columns=None):
        """
        Frequency encoding - thay thế bằng tần suất xuất hiện
        """
        df_copy = df.copy()
        
        if columns is None:
            cat_cols = df_copy.select_dtypes(include=['object', 'category']).columns
        else:
            cat_cols = columns
            
        for col in cat_cols:
            if col in df_copy.columns:
                # Tính tần suất
                freq = df_copy[col].value_counts(normalize=True)
                
                # Thay thế bằng tần suất
                df_copy[col] = df_copy[col].map(freq)
                
        return df_copy

In [18]:
def train_test_split(X, y, test_size=0.2, random_state=None, shuffle=True):
    """Chia dữ liệu thành tập train và test"""
    if isinstance(X, pd.DataFrame):
        X_np = X.values
    else:
        X_np = X.copy()
        
    if isinstance(y, pd.Series):
        y_np = y.values
    else:
        y_np = y.copy()
    
    if len(X_np) != len(y_np):
        raise ValueError("X và y phải có cùng số lượng mẫu")
    
    n_samples = len(X_np)
    n_test = int(n_samples * test_size)
    n_train = n_samples - n_test
    
    indices = np.arange(n_samples)
    
    if shuffle:
        if random_state is not None:
            np.random.seed(random_state)
        np.random.shuffle(indices)
    
    train_indices = indices[:n_train]
    test_indices = indices[n_train:]
    
    X_train = X_np[train_indices]
    X_test = X_np[test_indices]
    y_train = y_np[train_indices]
    y_test = y_np[test_indices]
    
    return X_train, X_test, y_train, y_test

def kfold_split(X, y, n_splits=5, random_state=None, shuffle=True):
    """Chia dữ liệu cho K-Fold Cross Validation"""
    # (Đã lược bớt phần convert type để code ngắn gọn, logic giữ nguyên)
    if isinstance(X, pd.DataFrame): X_np = X.values
    else: X_np = X.copy()
    
    if isinstance(y, pd.Series): y_np = y.values
    else: y_np = y.copy()
    
    n_samples = len(X_np)
    indices = np.arange(n_samples)
    
    if shuffle:
        if random_state is not None: np.random.seed(random_state)
        np.random.shuffle(indices)
    
    fold_size = n_samples // n_splits
    folds = []
    
    for i in range(n_splits):
        test_start = i * fold_size
        test_end = test_start + fold_size if i < n_splits - 1 else n_samples
        
        test_mask = np.zeros(n_samples, dtype=bool)
        test_mask[test_start:test_end] = True
        train_mask = ~test_mask
        
        X_train = X_np[train_mask]
        X_test = X_np[test_mask]
        y_train = y_np[train_mask]
        y_test = y_np[test_mask]
        
        folds.append((X_train, X_test, y_train, y_test))
    
    return folds

def describe_data(df):
    """Mô tả dữ liệu chi tiết"""
    print("=== THÔNG TIN DỮ LIỆU ===")
    print(f"Shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nMissing Values:\n{df.isnull().sum()}")
    print(f"\nSummary Statistics:\n{df.describe()}")
    return {'shape': df.shape}

def get_correlation(df):
    """Tính ma trận tương quan"""
    numeric_df = df.select_dtypes(include=[np.number])
    if numeric_df.empty: return None
    return numeric_df.corr()

def detect_outliers(df, method='iqr', threshold=1.5):
    """Phát hiện outliers"""
    numeric_df = df.select_dtypes(include=[np.number])
    if numeric_df.empty: return pd.DataFrame()
    
    outliers_info = {}
    
    if method == 'iqr':
        for col in numeric_df.columns:
            Q1 = numeric_df[col].quantile(0.25)
            Q3 = numeric_df[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = numeric_df[(numeric_df[col] < (Q1 - threshold * IQR)) | 
                                  (numeric_df[col] > (Q3 + threshold * IQR))]
            outliers_info[col] = len(outliers)
            
    elif method == 'zscore':
        for col in numeric_df.columns:
            mean_val = numeric_df[col].mean()
            std_val = numeric_df[col].std()
            if std_val > 0:
                z_scores = np.abs((numeric_df[col] - mean_val) / std_val)
                outliers = numeric_df[z_scores > threshold]
                outliers_info[col] = len(outliers)
    
    return pd.DataFrame.from_dict(outliers_info, orient='index', columns=['outlier_count'])

In [11]:
def describe_data(df):
    """Mô tả dữ liệu chi tiết"""
    print("=== THÔNG TIN DỮ LIỆU ===")
    print(f"Shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nData Types:")
    print(df.dtypes)
    print(f"\nMissing Values:")
    print(df.isnull().sum())
    print(f"\nSummary Statistics:")
    print(df.describe())
    print(f"\nMemory Usage: {df.memory_usage().sum() / 1024:.2f} KB")
    
    return {
        'shape': df.shape,
        'columns': list(df.columns),
        'missing': df.isnull().sum().to_dict(),
        'dtypes': df.dtypes.to_dict()
    }


def get_correlation(df):
    """Tính ma trận tương quan"""
    numeric_df = df.select_dtypes(include=[np.number])
    
    if numeric_df.empty:
        print("Không có cột số để tính correlation")
        return None
    
    corr_matrix = numeric_df.corr()
    return corr_matrix


def detect_outliers(df, method='iqr', threshold=1.5):
    """
    Phát hiện outliers
    method: 'iqr' hoặc 'zscore'
    """
    numeric_df = df.select_dtypes(include=[np.number])
    
    if numeric_df.empty:
        return pd.DataFrame()
    
    outliers_info = {}
    
    if method == 'iqr':
        # IQR method
        for col in numeric_df.columns:
            Q1 = numeric_df[col].quantile(0.25)
            Q3 = numeric_df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            outliers = numeric_df[(numeric_df[col] < lower_bound) | 
                                  (numeric_df[col] > upper_bound)]
            outliers_info[col] = len(outliers)
            
    elif method == 'zscore':
        # Z-score method
        for col in numeric_df.columns:
            mean_val = numeric_df[col].mean()
            std_val = numeric_df[col].std()
            
            if std_val > 0:
                z_scores = np.abs((numeric_df[col] - mean_val) / std_val)
                outliers = numeric_df[z_scores > threshold]
                outliers_info[col] = len(outliers)
    
    return pd.DataFrame.from_dict(outliers_info, orient='index', columns=['outlier_count'])

In [19]:
class DataPipeline:
    """Pipeline xử lý dữ liệu từ đầu đến cuối"""
    
    def __init__(self):
        self.scaler = None
        self.encoder = None
        self.encoding_maps = {}
        self.feature_names = None
        
    def fit(self, X, y=None, 
            normalize=True, scale_method='standard',
            handle_missing=True, missing_strategy='mean',
            encode_categorical=True, encode_method='label'):
        
        X_processed = X.copy()
        
        # 1. Xử lý missing values
        if handle_missing:
            if missing_strategy == 'mean':
                X_processed = MissingValueHandler.fill_mean(X_processed)
            elif missing_strategy == 'median':
                X_processed = MissingValueHandler.fill_median(X_processed)
            elif missing_strategy == 'mode':
                X_processed = MissingValueHandler.fill_mode(X_processed)
        
        # 2. Encode categorical
        if encode_categorical:
            if encode_method == 'label':
                X_processed, self.encoding_maps = CategoricalEncoder.label_encode(X_processed)
            elif encode_method == 'one_hot':
                X_processed = CategoricalEncoder.one_hot_encode(X_processed)
        
        self.feature_names = list(X_processed.columns)
        
        # 3. Chuẩn hóa dữ liệu (Lưu ý: Chỉ fit scaler trên tập train!)
        if normalize:
            if scale_method == 'standard':
                self.scaler = StandardScaler()
                X_processed = self.scaler.fit_transform(X_processed)
            elif scale_method == 'minmax':
                self.scaler = Normalizer()
                X_processed = self.scaler.fit_transform(X_processed)
        
        return X_processed
    
    def transform(self, X):
        """Áp dụng transformations đã fit (Dùng cho tập Test)"""
        X_processed = X.copy()
        
        # 1. Xử lý missing
        if hasattr(self, 'missing_strategy'): # (Giả định đơn giản, thực tế cần lưu strategy)
            X_processed = MissingValueHandler.fill_mean(X_processed)
        
        # 2. Encode categorical (Dùng map đã học)
        if self.encoding_maps:
            for col, mapping in self.encoding_maps.items():
                if col in X_processed.columns:
                    # Map giá trị, nếu ko thấy thì điền NaN hoặc xử lý riêng
                    X_processed[col] = X_processed[col].map(mapping)
                    # Fill NaN sinh ra do giá trị lạ trong tập test (nếu cần)
                    X_processed[col] = X_processed[col].fillna(-1) 
        
        # 3. Chuẩn hóa (Dùng scaler đã học từ train)
        if self.scaler:
            X_processed = self.scaler.transform(X_processed)
        
        return X_processed

In [22]:
def test_library():
    """Hàm test thư viện"""
    print("=== KIỂM TRA THƯ VIỆN XỬ LÝ DỮ LIỆU ===")
    
    # Tạo dữ liệu mẫu
    np.random.seed(42)
    
    data = {
        'age': [25, 30, np.nan, 35, 40, 45, 50, 55, 60, 65],
        'salary': [30000, 35000, 40000, np.nan, 50000, 55000, 60000, 65000, 70000, 75000],
        'experience': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'department': ['IT', 'HR', 'IT', 'Finance', 'HR', 'IT', 'Finance', 'HR', 'IT', 'Finance'],
        'education': ['Bachelor', 'Master', 'Bachelor', 'PhD', 'Master', 
                      'Bachelor', 'PhD', 'Master', 'Bachelor', 'PhD']
    }
    
    df = pd.DataFrame(data)
    print("1. Dữ liệu gốc:")
    print(df)
    print("\n2. Thông tin missing:")
    print(MissingValueHandler.count_missing(df))
    
    # Test xử lý missing values
    df_filled = MissingValueHandler.fill_mean(df, ['age', 'salary'])
    print("\n3. Sau khi fill missing với mean:")
    print(df_filled)
    
    # Test encode categorical
    df_encoded, mapping = CategoricalEncoder.label_encode(df_filled, ['department', 'education'])
    print("\n4. Sau khi label encode:")
    print(df_encoded)
    print("Mapping:", mapping)
    
    # Test chuẩn hóa
    scaler = StandardScaler()
    numeric_cols = ['age', 'salary', 'experience']
    df_numeric = df_encoded[numeric_cols]
    df_scaled = scaler.fit_transform(df_numeric)
    print("\n5. Sau khi standard scaling:")
    print(df_scaled)
    
    # Test split train/test
    y = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1])
    X_train, X_test, y_train, y_test = train_test_split(
        df_scaled, y, test_size=0.3, random_state=42
    )
    print("\n6. Kết quả split:")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    
    # Test pipeline
    pipeline = DataPipeline()
    X_processed = pipeline.fit(df, 
                              normalize=True,
                              scale_method='standard',
                              handle_missing=True,
                              missing_strategy='mean',
                              encode_categorical=True,
                              encode_method='label')
    print("\n7. Kết quả pipeline:")
    print(X_processed.head())
    
    print("\nKIỂM TRA HOÀN TẤT!")


if __name__ == "__main__":
    test_library()

=== KIỂM TRA THƯ VIỆN XỬ LÝ DỮ LIỆU ===
1. Dữ liệu gốc:
    age   salary  experience department education
0  25.0  30000.0           1         IT  Bachelor
1  30.0  35000.0           2         HR    Master
2   NaN  40000.0           3         IT  Bachelor
3  35.0      NaN           4    Finance       PhD
4  40.0  50000.0           5         HR    Master
5  45.0  55000.0           6         IT  Bachelor
6  50.0  60000.0           7    Finance       PhD
7  55.0  65000.0           8         HR    Master
8  60.0  70000.0           9         IT  Bachelor
9  65.0  75000.0          10    Finance       PhD

2. Thông tin missing:
age           1
salary        1
experience    0
department    0
education     0
dtype: int64

3. Sau khi fill missing với mean:
    age        salary  experience department education
0  25.0  30000.000000           1         IT  Bachelor
1  30.0  35000.000000           2         HR    Master
2  45.0  40000.000000           3         IT  Bachelor
3  35.0  53333.333333  